In [ ]:
"""
Project 3: Failure Mode and Effect Analysis (FMEA) for Financial Transactions
Objective: Develop a system to perform FMEA on financial transactions and visualize the results using a graph database.

Objectives 

    Develop an FMEA-Based Financial Analysis System – Implement Failure Modes and Effects Analysis (FMEA) to identify potential risks in financial transactions.
    
    Construct a Graph-Based Transaction Model – Represent financial transactions as a graph in Neo4j, with nodes as transactions and edges as relationships.
    
    Identify and Analyze Failure Modes – Detect potential failure points in transactions and assess their impact on financial operations.
    
    Compare Transactions Against Risk Benchmarks – Establish expected transaction patterns and identify deviations that indicate potential failures.
    
    Visualize and Report Insights – Generate interactive graphs and reports to highlight failure modes and improve financial risk management.

"""

In [1]:
!pip install gradio networkx matplotlib pandas numpy neo4j fpdf


In [ ]:
import gradio as gr
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from neo4j import GraphDatabase
from fpdf import FPDF
import tempfile

# Connect to Neo4j
URL = "bolt://localhost:7687"  
USERNAME = "neo4j"
PASSWORD = "password"

driver = GraphDatabase.driver(URL, auth=(USERNAME, PASSWORD))
print("Connected to Neo4j successfully!")

# Generate sample stock trade data with gaps
np.random.seed(42)
data = {
    "trade_id": range(1, 21),
    "ticker": ["AAPL"] * 10 + ["MSFT"] * 10,
    "volume": [1000, 1200, 0, 1100, 1300, 1400, 1250, 0, 1150, 1350,
               2000, 0, 2100, 2500, 2600, 2700, 2550, 2300, 0, 2450],
    "price": [150, 152, 180, 151, 153, 154, 149, 0, 150, 155,
              300, 0, 305, 310, 315, 318, 312, 290, 0, 295],
    "timestamp": pd.date_range("2024-02-01", periods=20, freq="h")
}

df = pd.DataFrame(data)

# Insert trades into Neo4j
def insert_trade(tx, trade):
    query = """
    CREATE (t:Trade {trade_id: $trade_id, ticker: $ticker, 
                     volume: $volume, price: $price, timestamp: $timestamp})
    """
    tx.run(query, **trade)

with driver.session() as session:
    for _, row in df.iterrows():
        session.write_transaction(insert_trade, row.to_dict())

print("Inserted stock trades into Neo4j!")

# Create relationships between trades
def create_relationships(tx):
    query = """
    MATCH (t1:Trade), (t2:Trade) 
    WHERE t1.timestamp < t2.timestamp
    CREATE (t1)-[:NEXT_TRADE]->(t2)
    """
    tx.run(query)

with driver.session() as session:
    session.write_transaction(create_relationships)

print("Created relationships between consecutive trades!")

# Identify gaps
df["missing_volume"] = df["volume"] == 0
df["missing_price"] = df["price"] == 0
df["price_change"] = df["price"].diff().fillna(0).abs()
df["sudden_jump"] = df["price_change"] > 20
df["is_gap"] = df["missing_volume"] | df["missing_price"] | df["sudden_jump"]

# Mark gaps in Neo4j
def mark_gaps(tx, trade_id, is_gap):
    query = """
    MATCH (t:Trade {trade_id: $trade_id})
    SET t.is_gap = $is_gap
    """
    tx.run(query, trade_id=int(trade_id), is_gap=bool(is_gap))

with driver.session() as session:
    for _, row in df.iterrows():
        session.write_transaction(mark_gaps, row["trade_id"], row["is_gap"])

print("Marked gaps in Neo4j!")

# Function to plot graphs
def plot_graphs():
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # Stock Price Trends
    colors = df["is_gap"].map({True: "red", False: "green"})
    axes[0].scatter(df["timestamp"], df["price"], c=colors, label="Price Points")
    axes[0].plot(df["timestamp"], df["price"], linestyle="-", marker="o", alpha=0.6)
    axes[0].set_xlabel("Timestamp")
    axes[0].set_ylabel("Price")
    axes[0].set_title("Stock Price Trends with Gaps")
    axes[0].grid(True)

    # Trade Network Graph
    G = nx.DiGraph()
    for _, row in df.iterrows():
        node_color = "red" if row["is_gap"] else "green"
        G.add_node(row["trade_id"], color=node_color)
    for i in range(len(df) - 1):
        G.add_edge(df.loc[i, "trade_id"], df.loc[i+1, "trade_id"])
    pos = nx.spring_layout(G, seed=42)
    node_colors = [G.nodes[n]["color"] for n in G.nodes]
    nx.draw(G, pos, with_labels=True, node_color=node_colors, edge_color="gray",
            node_size=800, font_size=10, font_weight="bold", arrowsize=12, ax=axes[1])
    axes[1].set_title("Stock Trade Network Graph")

    plt.tight_layout()
    return fig

# Function to generate PDF
def generate_pdf():
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=10)

    # Page 1: Graphs
    pdf.add_page()
    pdf.set_font("Arial", "B", 14)
    pdf.cell(200, 10, "Stock Trade Analysis Report", ln=True, align="C")
    pdf.ln(5)

    price_trends_path = tempfile.NamedTemporaryFile(delete=False, suffix=".png").name
    network_path = tempfile.NamedTemporaryFile(delete=False, suffix=".png").name

    fig = plot_graphs()
    fig.savefig(price_trends_path, dpi=100)
    plt.close(fig)

    pdf.image(price_trends_path, x=10, w=190)  # Fit both graphs on one page
    pdf.ln(10)

    # Page 2: Table & Summary
    pdf.add_page()
    pdf.set_font("Arial", "B", 9)
    pdf.cell(0, 10, "Trade Data Table:", ln=True)

    pdf.set_font("Arial", "", 9)
    column_width = 30
    headers = df.columns.tolist()
    
    for header in headers:
        pdf.cell(column_width, 8, header, border=1, align="C")
    pdf.ln()

    for _, row in df.iterrows():
        for value in row:
            pdf.cell(column_width, 8, str(value), border=1, align="C")
        pdf.ln()

    pdf.ln(10)
    pdf.set_font("Arial", "B", 12)
    pdf.cell(0, 10, "Summary", ln=True)

    pdf.set_font("Arial", "", 10)
    pdf.cell(0, 8, f"Total Trades: {len(df)}", ln=True)
    pdf.cell(0, 8, f"Total Gaps Detected: {df['is_gap'].sum()}", ln=True)
    pdf.cell(0, 8, f"Missing Volume Entries: {df['missing_volume'].sum()}", ln=True)
    pdf.cell(0, 8, f"Missing Price Entries: {df['missing_price'].sum()}", ln=True)

    pdf_path = tempfile.NamedTemporaryFile(delete=False, suffix=".pdf").name
    pdf.output(pdf_path)
    
    return pdf_path

# Gradio Interface
graph_component = gr.Plot(plot_graphs)
table_component = gr.DataFrame(df)
pdf_download = gr.File(generate_pdf, label="Download Report")

demo = gr.Interface(
    fn=None,
    inputs=[],
    outputs=[graph_component, table_component, pdf_download],
    live=False,
    title="Stock Trade Analysis",
    description="Analyze stock trade gaps with graphs, tables, and downloadable reports."
)

demo.launch()
